In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
api_key = 'Fqx6qkeb9Xd8heFqXGIsDqYhP2YOVwfOSV0skodn'
base_url = 'https://api.usa.gov/crime/fbi/cde/summarized/state'

states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA',
          'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD',
          'MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ',
          'NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
          'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY',
          'DC']

offenses = ['V','ASS','BUR','LAR','MVT','HOM','RPE','ROB','ARS','P']

offense_names = {
    'V'  : 'Violent Crime',
    'ASS': 'Assault',
    'BUR': 'Burglary',
    'LAR': 'Larceny',
    'MVT': 'Motor Vehicle Theft',
    'HOM': 'Homicide',
    'RPE': 'Rape',
    'ROB': 'Robbery',
    'ARS': 'Arson',
    'P'  : 'Property Crime'
}

all_data = []

for state in states:
    for offense in offenses:
        url = f"{base_url}/{state}/{offense}?from=01-2010&to=12-2023&api_key={api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            actuals = data.get('offenses', {}).get('actuals', {})
            populations = data.get('populations', {}).get('population', {})

            if actuals:
                state_name = list(actuals.keys())[0]
                monthly_crimes = actuals[state_name]
                monthly_pops = populations.get(state_name, {})

                for month_year, crime_count in monthly_crimes.items():
                    population = monthly_pops.get(month_year, None)
                    all_data.append({
                        'Month Year': month_year,
                        'State': state,
                        'State Name': state_name,
                        'Offense Code': offense,
                        'Offense Name': offense_names[offense],
                        'Crime Count': crime_count,
                        'Population': population
                    })

df = pd.DataFrame(all_data, columns = ['Month Year','State','State Name','Offense Code','Offense Name','Crime Count','Population'])
df['Date'] = pd.to_datetime(df['Month Year'], format='%m-%Y')
df['Year'] = pd.to_datetime(df['Date']).dt.year.astype(int)
df = df[['Date','Year','State Name','Offense Name','Crime Count','Population']]

final_df = df.groupby(['Year','State Name','Offense Name'], as_index = False).agg({'Crime Count' : 'sum', 'Population' : 'max'})
final_df.sort_values(by = ['State Name','Offense Name','Year'], inplace = True, ignore_index = True)

final_df.to_excel('crime_data_2010_2023.xlsx', sheet_name = 'Crime Data')
